# Joining and Appending DataFrames in PySpark HW

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [1]:
import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("joins").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


## Read in the database

Let cotinue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [2]:
import os

path = "dataset/madison_courses/uw-madison-courses/"

df_list = []
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filename_list = filename.split(".") #separate path from .csv
        df_name = filename_list[0]
        df = spark.read.csv(path+filename,inferSchema=True,header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + ' = df')
        
# QA
print("Full list of dfs:")
print(df_list)

Full list of dfs:
['courses', 'course_offerings', 'grade_distributions', 'instructors', 'rooms', 'schedules', 'sections', 'subjects', 'subject_memberships', 'teachings']


Now check the contents of a few of the dataframses that were read in above.

In [3]:
for df_name in df_list:
  print(f"DataFrame: {df_name}")
  print(globals()[df_name].show(5))  # Assuming DataFrames are stored in globals()


DataFrame: courses
+--------------------+--------------------+------+
|                uuid|                name|number|
+--------------------+--------------------+------+
|a3e3e1c3-543d-3bb...|Cooperative Educa...|     1|
|c070a84f-648f-351...|Cooperative Educa...|     1|
|e6b4b7ae-0e0b-3aa...|Cooperative Educa...|     1|
|8f63bde1-ff7f-3fe...|Workshop in Dance...|     1|
|f3541888-584a-392...|Cooperative Educa...|     1|
+--------------------+--------------------+------+
only showing top 5 rows

None
DataFrame: course_offerings
+--------------------+--------------------+---------+--------------------+
|                uuid|         course_uuid|term_code|                name|
+--------------------+--------------------+---------+--------------------+
|344b3ebe-da7e-314...|a3e3e1c3-543d-3bb...|     1092|Cooperative Educa...|
|f718e6cd-33f0-3c1...|a3e3e1c3-543d-3bb...|     1082|Cooperative Educa...|
|ea3b717c-d66b-30d...|a3e3e1c3-543d-3bb...|     1172|Cooperative Educa...|
|075da420-5f49

## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [13]:
# Assuming DataFrames are stored in globals()
from pyspark.sql.functions import col
rooms_df = globals()['rooms']
sections_df = globals()['sections']
course_offerings_df = globals()['course_offerings']

joined_sections_rooms = sections_df.join(rooms_df, sections_df.room_uuid == rooms_df.uuid, how="left")

# Join the course_offerings and sections tables on course_offering_uuid
joined_offerings_sections = course_offerings_df.join(joined_sections_rooms, course_offerings_df.uuid == sections_df.course_offering_uuid, how="inner")

# Select the desired columns

selected_df = joined_offerings_sections.select(
    col('room_uuid'), col('facility_code'), col('room_code'),
    col('term_code'), col('name')
)



# Show the results
selected_df.show()


+--------------------+-------------+---------+---------+--------------------+
|           room_uuid|facility_code|room_code|term_code|                name|
+--------------------+-------------+---------+---------+--------------------+
|                null|         NULL|     NULL|     1092|Cooperative Educa...|
|                null|         NULL|     NULL|     1082|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1172|Cooperative Educa...|
|                null|         NULL|     NULL|     1172|Cooperative Educa...|
|                null|         NULL|     NULL|     1172|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1172|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1114|Cooperative Educa...|
|                null|         NULL|     NULL|     1114|Cooperative Educa...|
|                null|         NULL|     NULL|     1104|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1104|Cooperat

## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [15]:
selected_df = joined_offerings_sections.select(
    col('room_uuid'), col('facility_code'), col('room_code'),
    col('term_code'), col('name')
).where(col('facility_code') == '0469')

selected_df.show()

+--------------------+-------------+---------+---------+--------------------+
|           room_uuid|facility_code|room_code|term_code|                name|
+--------------------+-------------+---------+---------+--------------------+
|9759cb5f-a7d3-3d0...|         0469|     2441|     1152|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1092|  Fundamentals-Flute|
|6af80b0b-b3e3-370...|         0469|     4411|     1172|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1162|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1132|  Fundamentals-Flute|
|50322d30-dd8f-3c6...|         0469|     2511|     1072|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1142|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1112|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1102|  Fundamentals-Flute|
|50322d30-dd8f-3c6...|         0469|     2511|     1082|  Fundam

## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [ ]:
values = [('Monkey',10),('Monkay',36),('Mnky',123), \
          ('Elephant',48),('Elefant',16),('Ellafant',1), \
          ('Hippopotamus',48),('Hipopotamus',16),('Hippo',1)]
zoo = spark.createDataFrame(values,['Animal','age'])
zoo.show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkay| 36|
|        Mnky|123|
|    Elephant| 48|
|     Elefant| 16|
|    Ellafant|  1|
|Hippopotamus| 48|
| Hipopotamus| 16|
|       Hippo|  1|
+------------+---+



### Great job! 